This is an application that attempts to simulate the game of k-NIM in the case k = 2. 
feature additions will employ Monte Carlo Methods as we attempt to understand game dynamics further.

Our next stage is databuilding through implementing a SQL server and testing the mondel. 

Credits:  
        John Tan Chong Min for his python tutorials and his work in writing a UCT / MCTS for classical NIM. 
        Gelly and Silver for there work on MC-Rave in Computer GO



In [1055]:
#libraries that are used in the program
from copy import deepcopy
import numpy as np
import random 
from treelib import Node, Tree

In [1056]:
#helpers to avoid duplicates 

#check if two arrays are unordered matches:
def is_match(a,b): 
    match = True
    for item in a: 
        if a.count(item) != b.count(item):
            match = False
            break
    return match

#check if an array has such a match in a collection of other arrays 
def occurs(collection, element):
    has_element = False
    for item in collection:
        if is_match(item, element):
            has_element = True
            break

    return has_element

def all_even(vector):
    result = True
    for x in vector: 
        if x % 2  == 1 :
            result = False
            break
    return result

def as_str(array): 
    string = ''.join(str(char) for char in array)
    return string 
 

In [1057]:
class two_NIM:

    def __init__(self, position):
        self.position = position 
        self.sum = sum(position) 
        self.in_play = True
        self.turn = 1
        self.descendants = self.plays()
        self.reward = 1

    def move(self, play):

        assert(play in self.descendants) 

        self.position = play #update position
        self.sum = sum(play) #update sum 
        self.descendants = self.plays() #update descendants

        if self.over() == True:  #check loss
            self.in_play = False #end game
            if(self.turn == - 1):
                self.reward = 0  #punish losers

        self.turn = self.turn * -1  # update turn
        

    def over(self):
        return (self.sum == 0) # game ends when sum accross all heaps is zero

    def reset(self):
        self.sum = sum(self.position)
        self.in_play = True
        self.reward = 1
        self.turn = 1

    def plays(self): 
        #calculate all descendants: (currently we remove unordered duplicates)
        moves = []
        for index, x in enumerate(self.position):
            child = deepcopy(self.position)
            if(x > 0):
                child[index] -= 1
                if not occurs(moves, child):
                    moves.append(child)

        for indexx, x in enumerate(self.position):
            for indexy, y in enumerate(self.position):
                child = deepcopy(self.position)
                if((x > 0) and (y > 0) and (indexx != indexy)): 
                    child[indexx] -= 1
                    child[indexy] -= 1 
                    if not occurs(moves, child):
                        moves.append(child)
        
        return moves 

In [1058]:
#next, we need to make a game method to play the actual games
def Game(P_1, P_2, position, with_script):
    
    game = two_NIM(position)

    while(game.in_play):
        if (game.turn == 1):
            #print(game.descendants())
            move = P_1(game.descendants)
            game.move(move)
            if(with_script):
                print(f"Player 1 moves to {move}")
        else:
            move = P_2(game.descendants)
            game.move(move)
            if(with_script):
                print(f"Player 2 moves to {move}")
        
    if game.turn == 1 : 
        winner = 2
    else :
        winner = 1

    print(f"Player {winner} wins")

    return game.reward

In [1059]:
# a random player just selects a random position from an array of positions. 
def random_player(position):
    return position[random.randint(0, len(position)-1)]

In [1060]:
# now that we ahve a game method, we can play tournaments
def Tournament(P_1, P_2, position, rounds, bool): 

    P_1_score = P_2_score = 0 

    for i in range(0, rounds): 
        result = Game(P_1, P_2, position, bool)
        if(result == 1): 
            P_1_score += 1
            #P_2_score -= 1
        else: 
           # P_1_score -=1
            P_2_score +=1

    print(f"Player 1 wins {P_1_score} games.")
    print(f"Player 2 wins {P_2_score} games.")

    return [f"Player 1 wins {P_1_score} games , Player 2 wins {P_2_score} games"]
    


In [1061]:
#play a random game and record the reward [1 if win, -1 if loss]
def random_expirament(position):
    if (sum(position) == 0): #just in case we start empty
        return 1
    else: 
        return Game(random_player,random_player, position, True)

In [1062]:
def mc_player(position, iterations):
     #iterations = number of times you want to perform the random expirament
     # more iterations means more accurate weights, at the expense of efficiency.
    game = two_NIM(position)
    weights = {} 
    output = []
    for choice in game.descendants: # for each choice (move) in the decisions space (position)
        outcome = 0 
        for x in range(iterations): 
            #print(choice)
            outcome += -(random_expirament(choice)) # play the game randomly
            # it's necessary to factor out the -1 because if a choice is a "good position," then you DON'T want to move there. 
            #print(outcome)

        weights[game.descendants.index(choice)] = outcome/iterations #assign a weight to the choice 
        output.append([choice, weights[game.descendants.index(choice)]])
        
    #print(weights)
    return output

In [1063]:
print(Game(random_player,random_player, [3,3,3], True))
#print(random_expirament([3,3,3]))
#print(mc_player([2,1], 100))
#print(random_player([[1,0],[1,1],[2,0]]))
#print(mc_player([2,5,5], 100))
#print(mc_player([2,1], 10))
#Tournament(random_player,random_player, [3,3,3], 10, False)

Player 1 moves to [2, 3, 3]
Player 2 moves to [2, 2, 2]
Player 1 moves to [1, 1, 2]
Player 2 moves to [0, 1, 2]
Player 1 moves to [0, 0, 2]
Player 2 moves to [0, 0, 1]
Player 1 moves to [0, 0, 0]
Player 1 wins
1


In [1064]:
# a state (i.e. game state) can be thought of as a node in the game tree
# each node has a parent and a set of children
class State: 
    
    def __init__(self, position): 
            #self.parent = parent #parent node
            self.position = position #game state as a position 
            #self.children = [] # the children of the node in the game tree
            self.count = 0
            self.value = 0

    def update(self, reward): #update the value and count of the node
          self.count += 1
          self.value = (self.value + reward)/self.count

In [1065]:
game_tree = Tree()

game_tree.create_node(None, "root", data = State([1,2,3,]))

node = game_tree.get_node("root")

print(node.identifier, node.data.position)

node.data.position = [0,0,0]

game_tree.add_node
print(node.identifier, node.data.position)
node.data.update(1)
print(node.identifier, node.data.count)
print(node.identifier, node.data.value)
node.data.update(0)
print(node.identifier, node.data.count)
print(node.identifier, node.data.value)


root [1, 2, 3]
root [0, 0, 0]
root 1
root 1.0
root 2
root 0.5


Implementing greedy monte coarlo with amaf. 

the basic assumption that makes this algorithm effective is that any move in two_NIM is equally good wherever it is played in the game tree. 


In [1094]:
def BACKUP(game_tree: Tree, node: Node, reward): 

    #we go through the tree and update all the nodes 
    t = 0
    while not node.identifier == "root" :

        print(f"node: {node.identifier}")
        #print(f"value: {node.data.value}, count:{node.data.count}")

        if t % 2 == 0:
            node.data.update(0)
            #print(f"updated value: {node.data.value}, updated count:{node.data.count}")
            node = game_tree.parent(node.identifier)
            
        else : 
            node.data.update(1) 
            #print(f"updated value: {node.data.value}, updated count:{node.data.count}")
            node = game_tree.parent(node.identifier)
        t += 1

    print(f"node: {node.identifier}")
    #print(f"value: {node.data.value}, count:{node.data.count}")


    if t % 2 == 0:
            node.data.update(0)
            #print(f"updated value: {node.data.value}, updated count:{node.data.count}")
            node = game_tree.parent(node.identifier)
            
    else : 
        node.data.update(1) 
        #print(f"updated value: {node.data.value}, updated count:{node.data.count}")
        node = game_tree.parent(node.identifier)

    



In [1067]:
# as we learn more about the game, we can update default_policy to improve performance. 
# At the moment, the default policy is just random selection. 

#initial debugging complete 

def default_policy(game):
    return random_player(game.descendants)

In [1068]:
#evaluate each possible move and select the best or worst one depending on if it is your turn or not. 
def best_move(game_tree, ID, turn): 

    best_kiddo = None
    max_value = 0
    worst_kiddo = None
    min_value = 0

    for kiddo in game_tree.children(ID):
        if kiddo.data.value >=  max_value: 
            best_kiddo = kiddo
            max_value = kiddo.data.value
        if kiddo.data.value <=  min_value: 
            worst_kiddo = kiddo
            min_value = worst_kiddo.data.value

    if turn == 1 :
        return best_kiddo
    else:
        return worst_kiddo

In [1082]:
# a simulation in which both players follow the default policy in every move
# at the moment, this is pure Monte Carlo

#initial debugging complete

def default_sim(game: two_NIM, game_tree: Tree): 

    node = game_tree.get_node("root")  # to track the nodes

    while game.in_play: 

        next_move = default_policy(game) # select a move based on the default policy
        ID = as_str(next_move)

        print(next_move)

        if not game_tree.contains(ID): 
            #when the node is not in the tree, we add a new one
            ndata = State(next_move)
            new_node = game_tree.create_node(None, ID, parent = node.identifier, data = ndata)
            node = new_node
        else:
            node = game_tree.get_node(ID) # move to the node 
                      
        game.move(next_move) #move the game into the next position

    BACKUP(game_tree, node, game.reward) #update the tree

In [1070]:
def tree_sim(game: two_NIM, game_tree: Tree): 
    
    node =  game_tree.get_node("root")

    while game.in_play : 
        
        ID = as_str(game.position) #to locate the nodes in the backup phase

        print("executed")

        if not game_tree.contains(ID): 
            #when the node is not in the tree, we create a new one for the position new one, select a random branch, and continue on
            ndata = State(game.position) 
            new_node = game_tree.create_node(None, ID, parent = node.identifier, data = ndata)
            node = new_node
            next_move = default_policy(game)
            print(game.position)
            print(next_move)
            game.move(next_move) 

        else: 
            # when the node is in the tree, we select the best option already there
            node = game_tree.get_node(ID)
            next_move = best_move(game_tree, ID, game.turn).data.position
            print(game.position)
            print(next_move)
            game.move(next_move)

    BACKUP(game_tree, node, game.reward) #bakck-propogation

In [1071]:
def simulate(position, iterations): 

    #we make a new game tree with position as the root
    game_tree = Tree() 
    game_tree.create_node(None, "root", None, State(position))

    while iterations >= 0 :
        
        game = two_NIM(position) # simulate a game
        default_sim(game, game_tree) #one default simualtion: this helps mitigate selection bias as well as build out the tree
        game = two_NIM(position)
        tree_sim(game, game_tree)    #one biased simulation

        iterations -= 1

    


    return best_move(game_tree, "root", 1).data.position



In [1095]:
game = two_NIM([3,3,4])
game_tree = Tree() 
game_tree.create_node(None, "root", None, State(game.position))

default_sim(game, game_tree)
game = two_NIM([3,3,4])
default_sim(game, game_tree)


game_tree.depth()

node = game_tree.get_node("000")

#BACKUP(game_tree,node, 0)

#print(default_policy(game))

# it is taking a default path in the backup phase: the backup phase needs to be started from the previous node.



[2, 2, 4]
[2, 2, 3]
[1, 2, 2]
[0, 1, 2]
[0, 0, 2]
[0, 0, 1]
[0, 0, 0]
node: 000
node: 001
node: 002
node: 012
node: 122
node: 223
node: 224
node: root
[3, 3, 3]
[2, 2, 3]
[1, 1, 3]
[0, 1, 2]
[0, 1, 1]
[0, 0, 0]
node: 000
node: 001
node: 002
node: 012
node: 122
node: 223
node: 224
node: root
